<h3><center>Black Scholes Formula<br>
CUNY SPS MSDS<br>
Professor Samuel Gralnick<br>
Team Sugar Cane: Euclid Zhang, Jie Zou, Zhenni Xie<br>
Fall 2022</center></h3>

In this analysis, we calculate the prices of the european call options and european put options of the equities in our investment portfolio.

In [8]:
#Loading packages

import numpy as np
from datetime import datetime, timedelta
import yfinance as yf
import pandas_datareader.data as pdr
import pandas as pd
import scipy.stats as stats
import math

In the calculations, we calcualte the returns and volatilities using the assets' prices for 1 calendar year. The risk free rate is from https://www.marketwatch.com/investing/bond/tmubmusd01y?countrycode=bx. We assume that there are 251 trade days in a calendar year, this is used to annualized the returns and standard deviations of the assets. We will set the strike price equal to 80% of the current price for the option calculations. We will calculate the prices of option with expiration period 1 calendar year.

In [9]:
#The start date and end date of the assets' prices used for the calculations

start = datetime.strptime('2021-10-08', '%Y-%m-%d')
end = datetime.strptime('2022-10-08', '%Y-%m-%d')

#Seting up parameters for the option calculations. 

#1-year risk free rate from https://www.marketwatch.com/investing/bond/tmubmusd01y?countrycode=bx
risk_free_rate = 0.04538
#251 trade days in a year, used to annualized the return and standard deviation
n_trade_days = 251
#percentage of the current price, used as the strike price of the options
strike_price_pct = 0.8
#Expiration period of the option, 1 year
time_period = 1

Our investment portfolio includes the following assets, with purchasing prices (close prices) from 9/16/2022. In this analysis, **we will not include the U.S. 30 Year Treasury bonds in our calculation**

In [10]:
#Load the infromation of the assets in our portfolio

assets = pd.read_csv("portfolio assets.csv")
assets

,Ticker,Type,Sector,Price,Units,Amount
0,US30Y,U.S. 30 Year Treasury,NaN,90.44,467317.00,42262981.19
1,HSON,Common Stock,COMMERCIAL SERVICES,34.95,36138.24,1263031.49
2,GOGO,Common Stock,COMMUNICATIONS,13.32,94822.19,1263031.57
3,JAKK,Common Stock,CONSUMER DURABLES,23.37,54044.99,1263031.42
4,CALM,Common Stock,CONSUMER NON-DURABLES,56.85,22216.91,1263031.33
5,HRB,Common Stock,CONSUMER SERVICES,44.93,28111.09,1263031.27
6,HDSN,Common Stock,DISTRIBUTION SERVICES,7.94,159071.98,1263031.52
7,BELFA,Common Stock,ELECTRONIC TECHNOLOGY,28.52,44285.81,1263031.30
8,ARLP,Common Stock,ENERGY MINERALS,23.88,52890.76,1263031.35
9,CI,Common Stock,HEALTH SERVICES,290.35,4350.03,1263031.21


We then retrieve the prices and volumes of the common stocks and ETFs in our portfolio, using the yfinance package

In [11]:
#retrieve the prices and volumes of the common stocks and ETFs in our portfolio, using the yfinance package

yf.pdr_override()

tickers = list(assets['Ticker'])[1:]

historical_data = pdr.DataReader(tickers, start, end , data_source='yahoo')

historical_data = historical_data.reset_index()
historical_data.Date = historical_data.Date.apply(datetime.date)
historical_data.set_index('Date', inplace = True)

close_prices = historical_data['Adj Close']
close_prices

[*********************100%***********************]  29 of 29 completed


,ARLP,ASC,AZPN,BELFA,BSM,CALM,CF,CI,CSL,DIG,...,JAKK,LNG,MUSA,SRTS,TBT,TMV,TTT,TYO,UUP,YCS
Date,,,,,,,,,,,,,,,,,,,,,
2021-10-08,10.953591,3.94,141.550003,14.275089,11.343108,34.127800,60.054222,201.523483,202.984573,19.716621,...,11.980000,98.628128,168.098984,3.86,18.889999,65.190002,36.779999,8.77,25.270000,39.599998
2021-10-11,11.000162,3.94,159.000000,13.684396,11.252582,34.009781,61.116436,200.295761,202.954849,19.582832,...,11.840000,100.567635,162.938202,3.89,19.000000,65.669998,37.099998,8.86,25.340000,40.384998
2021-10-12,11.046733,3.75,164.029999,13.261065,11.089631,34.275326,59.896858,198.498367,203.311584,19.624950,...,11.840000,101.448334,160.939499,3.92,18.370001,62.410000,35.169998,8.75,25.370001,40.544998
2021-10-13,10.795248,3.68,166.000000,13.054323,11.062473,33.970440,59.355915,195.915237,205.719589,19.669546,...,12.570000,103.912300,162.351517,3.93,18.020000,60.509998,34.099998,8.66,25.230000,40.305000
2021-10-14,10.702105,3.72,167.960007,13.647700,11.315950,34.039288,59.542793,199.421631,211.912964,20.137810,...,11.940000,104.862267,166.498047,4.08,17.870001,59.860001,33.730000,8.61,25.230000,40.584999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-03,23.620001,9.37,243.190002,28.760881,16.230000,55.029999,100.300003,287.220001,289.730011,33.919998,...,19.570000,167.330002,283.339996,13.13,30.809999,127.760002,72.480003,13.11,30.010000,65.059998
2022-10-04,24.129999,9.20,250.050003,28.631149,16.750000,55.230000,105.059998,293.019989,296.470001,36.860001,...,21.000000,170.419998,283.670013,12.66,30.969999,128.539993,73.000000,12.98,29.600000,64.440002
2022-10-05,24.480000,9.25,256.230011,28.740923,17.100000,55.869999,102.550003,294.029999,297.489990,38.250000,...,20.260000,171.440002,280.510010,12.10,31.629999,132.630005,75.370003,13.40,29.879999,64.989998


In the option price calculations, we use the log returns

In [12]:
#Calculate the daily log returns

log_return = np.log(close_prices / close_prices.shift(1)).dropna()

In [13]:
#Define a function to calculate the call option

def european_call(stock_price, strike_price, risk_free_rate, time_period, volatility):
  d_1 = (math.log(stock_price/strike_price) + (risk_free_rate + 0.5*(volatility**2))*time_period)/(volatility*math.sqrt(time_period))
  d_2 = d_1 - volatility*math.sqrt(time_period)
  option_price = stock_price * stats.norm.cdf(d_1) - strike_price * math.exp(-risk_free_rate*time_period) * stats.norm.cdf(d_2)
  return option_price

In [14]:
#Define a function to calculate the put option

def european_put(stock_price, strike_price, risk_free_rate, time_period, volatility):
  d_1 = (math.log(stock_price/strike_price) + (risk_free_rate + 0.5*(volatility**2))*time_period)/(volatility*math.sqrt(time_period))
  d_2 = d_1 - volatility*math.sqrt(time_period)
  option_price =  strike_price * math.exp(-risk_free_rate*time_period) * stats.norm.cdf(-d_2) - stock_price * stats.norm.cdf(-d_1)
  return option_price

In [15]:
#Gather the information for the option price calculation for each asset

asset_info = pd.DataFrame({"stock_price" : close_prices.iloc[len(close_prices)-1],
              "strike_price" : close_prices.iloc[len(close_prices)-1] * strike_price_pct,
              "risk_free_rate" : math.log(1+risk_free_rate),
              "time_period" : time_period,
              "volatility" : math.sqrt(n_trade_days) * log_return.std()})

In [16]:
#Calculate the prices of the call and put options

asset_info['call_price'] = asset_info.apply(lambda x: european_call(x['stock_price'], 
                                         x['strike_price'],
                                         x['risk_free_rate'],
                                         x['time_period'],
                                         x['volatility']), axis=1)
asset_info['put_price'] = asset_info.apply(lambda x: european_put(x['stock_price'], 
                                         x['strike_price'],
                                         x['risk_free_rate'],
                                         x['time_period'],
                                         x['volatility']), axis=1)

The following is the result of our calculations.

In [17]:
#Result of the calculations

asset_info

,stock_price,strike_price,risk_free_rate,time_period,volatility,call_price,put_price
ARLP,24.309999,19.448000,0.041411,1,0.486676,7.535796,1.884889
ASC,9.530000,7.624000,0.041411,1,0.601567,3.281386,1.066119
AZPN,253.410004,202.728003,0.041411,1,0.378462,70.809754,11.904109
BELFA,28.970451,23.176361,0.041411,1,0.489945,9.008223,2.273986
BSM,17.360001,13.888000,0.041411,1,0.345898,4.702079,0.666714
CALM,53.630001,42.904001,0.041411,1,0.314405,14.103900,1.637502
CF,104.800003,83.840002,0.041411,1,0.485397,32.447478,8.086514
CI,288.380005,230.704004,0.041411,1,0.268046,72.785361,5.750871
CSL,292.450012,233.960010,0.041411,1,0.294652,75.542032,7.561461
DIG,38.810001,31.048001,0.041411,1,0.693684,14.453360,5.431901


We then calculate the the ratio of option price to the current asset price

We observe that for a call option with strike price 20% lower than the current asset price, the option price is more than 
20% of the current asset price. This is reasonalbe since the call option is in the money and the option price must be higher than the asset price - strike price as of now.

The price of the put options are lower than the call options since the call options are out of the money.

The option price of a highly volatile asset is extremly expansive. For example, the price of the call option for HUDI is more than 50% of the current asset price and the price of the put option is more than 30% of the current asset price. We may consider selling the HUDI stocks and allocate the fund to less risky assets.

In [18]:
#Calculate the ratio of option price to the current asset price

asset_info['stike_to_price'] = asset_info['strike_price'] / asset_info['stock_price']
asset_info['call_to_price'] = asset_info['call_price'] / asset_info['stock_price']
asset_info['put_to_price'] = asset_info['put_price'] / asset_info['stock_price']
asset_info

,stock_price,strike_price,risk_free_rate,time_period,volatility,call_price,put_price,stike_to_price,call_to_price,put_to_price
ARLP,24.309999,19.448000,0.041411,1,0.486676,7.535796,1.884889,0.8,0.309987,0.077536
ASC,9.530000,7.624000,0.041411,1,0.601567,3.281386,1.066119,0.8,0.344322,0.111870
AZPN,253.410004,202.728003,0.041411,1,0.378462,70.809754,11.904109,0.8,0.279428,0.046976
BELFA,28.970451,23.176361,0.041411,1,0.489945,9.008223,2.273986,0.8,0.310945,0.078493
BSM,17.360001,13.888000,0.041411,1,0.345898,4.702079,0.666714,0.8,0.270857,0.038405
CALM,53.630001,42.904001,0.041411,1,0.314405,14.103900,1.637502,0.8,0.262985,0.030533
CF,104.800003,83.840002,0.041411,1,0.485397,32.447478,8.086514,0.8,0.309613,0.077161
CI,288.380005,230.704004,0.041411,1,0.268046,72.785361,5.750871,0.8,0.252394,0.019942
CSL,292.450012,233.960010,0.041411,1,0.294652,75.542032,7.561461,0.8,0.258308,0.025856
DIG,38.810001,31.048001,0.041411,1,0.693684,14.453360,5.431901,0.8,0.372413,0.139961
